<a href="https://colab.research.google.com/github/ghadirN/AI_Agent_Assignment/blob/main/the-cozy-oven-bot/business_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Cozy Oven

installing

In [1]:
!pip install openai python-dotenv gradio

importing libraries and loading all needed enviroments and files

In [2]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from google.colab import userdata

# --- MODIFIED FOR OPENROUTER ---
# This bypasses region blocks and lets you use Llama 3.3 for free/cheap
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=userdata.get("OPENROUTER_API_KEY"), # Add this key to Colab Secrets
)
# Using a 2026 flagship model that is excellent with tools
MODEL_NAME = "meta-llama/llama-3.3-70b-instruct"
# -------------------------------

# Paths for existing business files
SUMMARY_PATH = os.path.join("me", "business_summary.txt")
PDF_PATH = os.path.join("me", "about_business.pdf")

summary_text = ""
if os.path.exists(SUMMARY_PATH):
    with open(SUMMARY_PATH, "r", encoding="utf-8") as f:
        summary_text = f.read()
else:
    summary_text = (
        "The Cozy Oven is a small-batch, artisanal bakery that embodies slow, mindful moments."
    )

# Define where the tools will log their data
LEADS_LOG_PATH = "leads_log.jsonl"
FEEDBACK_LOG_PATH = "feedback_log.jsonl"


# import os
# import json
# from datetime import datetime

# from dotenv import load_dotenv
# from openai import OpenAI
# import gradio as gr

# # Get Access to The OpenAI API Key
# from getpass import getpass
# from google.colab import userdata

# open_ai_key = userdata.get('OPENAI_API_KEY')
# client = OpenAI(api_key=open_ai_key)

# # Paths for existing business files
# SUMMARY_PATH = os.path.join("me", "business_summary.txt")
# PDF_PATH = os.path.join("me", "about_business.pdf")  # not read, just assumed to exist


# summary_text = ""
# if os.path.exists(SUMMARY_PATH):
#     with open(SUMMARY_PATH, "r", encoding="utf-8") as f:
#         summary_text = f.read()
# else:
#     summary_text = (
#         "The Cozy Oven is a small-batch, artisanal bakery that embodies slow, mindful moments."
#     )

# # Define where the tools will log their data
# LEADS_LOG_PATH = "leads_log.jsonl"
# FEEDBACK_LOG_PATH = "feedback_log.jsonl"

In [3]:
# A function used to save data into a JSON Lines file
def _append_jsonl(path: str, record: dict) -> None:
    os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

# Tool 1: Collecting customer info
def record_customer_interest(email: str, name: str, message: str) -> str:
    """Log a potential customer lead with contact info and notes."""
    entry = {
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "name": name,
        "email": email,
        "message": message,
    }
    _append_jsonl(LEADS_LOG_PATH, entry)
    print("[LEAD RECORDED]", entry)
    return "Lead recorded successfully. Thank you for your interest in The Cozy Oven!"

# Tool 2: Recording what we don't know
def record_feedback(question: str) -> str:
    """Log a question or feedback when the agent cannot fully answer."""
    entry = {
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "question": question,
    }
    _append_jsonl(FEEDBACK_LOG_PATH, entry)
    print("[FEEDBACK RECORDED]", entry)
    return "Thanks for your feedback! The Cozy Oven team will review this."

System prompts

In [4]:
# System prompt for the Cozy Oven agent
system_prompt = f"""
You are the friendly assistant for **The Cozy Oven**, a slow-life artisanal bakery.

Use the business documents and your general baking knowledge to answer questions about:
- The Cozy Oven's mission, services (Daily Lamination, Hearth Table, Bread Subscriptions), and atmosphere.
- The team: Master Baker Ghadir and Creative Director Fatima.
- The unique value proposition: slow, unhurried, home-away-from-home bakery experience.

You have two important responsibilities:
1. Answer questions about The Cozy Oven: menu, vibe, services, ideal customers, and how things work.
2. Encourage and log leads:
   - If someone sounds interested in catering, subscriptions, or planning a visit,
     kindly invite them to share their **name**, **email**, and a short note.
   - When they share it, call `record_customer_interest`.

If you do not know the answer to a question, or if it is outside The Cozy Oven's scope,
explain that you will pass it to the human team and call `record_feedback` with the full question.

Business summary (from file):

{summary_text}
"""


business_tools = [
    {
        "type": "function",
        "function": {
            "name": "record_customer_interest",
            "description": "Record a new lead for The Cozy Oven, including contact info and what they are interested in.",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {
                        "type": "string",
                        "description": "Customer email address"
                    },
                    "name": {
                        "type": "string",
                        "description": "Customer full name"
                    },
                    "message": {
                        "type": "string",
                        "description": "Short note about what they want from The Cozy Oven"
                    },
                },
                "required": ["email", "name", "message"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": "Record customer feedback or a question the assistant could not fully answer.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The customer's question or feedback."
                    },
                },
                "required": ["question"],
            },
        },
    },
]

# Mapping from tool name to Python function
BUSINESS_TOOL_FUNCS = {
    "record_customer_interest": record_customer_interest,
    "record_feedback": record_feedback,
}

Building the logic of the agent

In [5]:
def run_business_agent(user_input: str, chat_history=None, max_steps: int = 5) -> str:
    messages = [{"role": "system", "content": system_prompt}]
    if chat_history:
        for u, a in chat_history:
            messages.append({"role": "user", "content": u})
            messages.append({"role": "assistant", "content": a})
    messages.append({"role": "user", "content": user_input})

    for _ in range(max_steps):
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            tools=business_tools,
            tool_choice="auto",
        )

        message = response.choices[0].message

        # If the AI just speaks, return the text
        if not message.tool_calls:
            return message.content

        # If the AI requests tools, we MUST execute them and tell the AI the result
        messages.append(message)
        for tool_call in message.tool_calls:
            tool_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            # Execute your actual python function
            tool_func = BUSINESS_TOOL_FUNCS.get(tool_name)
            result = tool_func(**args) if tool_func else f"Tool {tool_name} not found."

            # Send result back to AI
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })

        # Ask the AI for a final natural response after it knows the tool worked
        final_response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages
        )
        return final_response.choices[0].message.content


# def run_business_agent(user_input: str, chat_history=None, max_steps: int = 5) -> str:
#     """Run The Cozy Oven agent with optional chat history and tool use."""
#     messages = [{"role": "system", "content": system_prompt}]

#     if chat_history:
#         for u, a in chat_history:
#             if u:
#                 messages.append({"role": "user", "content": u})
#             if a:
#                 messages.append({"role": "assistant", "content": a})

#     messages.append({"role": "user", "content": user_input})

#     for _ in range(max_steps):
#         response = client.chat.completions.create(
#             model="gpt-4.1-mini",
#             messages=messages,
#             tools=business_tools,
#             tool_choice="auto",
#         )

#         message = response.choices[0].message

#         if not message.tool_calls:
#             return message.content

#         messages.append(message)

#         for tool_call in message.tool_calls:
#             tool_name = tool_call.function.name
#             args = json.loads(tool_call.function.arguments)

#             tool_func = BUSINESS_TOOL_FUNCS.get(tool_name)
#             if tool_func is None:
#                 tool_result = f"Unknown tool: {tool_name}"
#             else:
#                 tool_result = tool_func(**args)

#             messages.append(
#                 {
#                     "role": "tool",
#                     "tool_call_id": tool_call.id,
#                     "content": tool_result,
#                 }
#             )


#         followup = client.chat.completions.create(
#             model="gpt-4o-mini",
#             messages=messages,
#         )
#         final_message = followup.choices[0].message

#         if not final_message.tool_calls:
#             return final_message.content

#         messages.append(final_message)

#     return "I'm having trouble completing your request right now, but I've logged it for the Cozy Oven team."

Gradio Interface

In [6]:
def gradio_business_chat(user_message, history):
    """Adapter between Gradio's (message, history) and our business agent."""
    reply = run_business_agent(user_message, chat_history=history)
    return reply


demo = gr.ChatInterface(
    fn=gradio_business_chat,
    title="The Cozy Oven Assistant",
    description=(
        "Chat with The Cozy Oven, a slow-life artisanal bakery. "
        "Ask about our croissants, Hearth Table catering, or Flourish Locals bread subscriptions. "
        "If you're interested in working with us, the assistant will politely ask for your name and email."
    ),
)

# In a notebook, just call demo.launch()
demo.launch()

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6de92a8b7049a74043.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
